# Importación de dependencias y lectura de datasets.

In [279]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import OneHotEncoder, MinMaxScaler

from tensorflow.keras.layers import Input, Dense, Concatenate, LeakyReLU
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Dataset Oracle-RedBull
races_info = pd.read_csv('final_data_abu_dhabi.csv')

# Definición de variables de entrada y salida

In [280]:
# Variables de entrada (condiciones)
input_cols = [
    'EventName', 'RoundNumber', 'eventYear', 'Team', 'Driver',
    'meanAirTemp', 'meanTrackTemp', 'meanHumid', 'Rainfall',
    'GridPosition', 'CircuitLength', 'designedLaps'
]

# Variables de salida (estrategia)
output_cols = ['lapNumberAtBeginingOfStint', 'Compound', 'StintLen']

# EDA

In [281]:
# Limpieza nulos
races_info = races_info[input_cols + output_cols].dropna()

In [282]:
# Codificar variables categóricas de entrada

categorical_cols_input = ['EventName', 'Team', 'Driver', 'Rainfall']
encoder_input = OneHotEncoder(sparse_output=False)
encoded_input = encoder_input.fit_transform(races_info[categorical_cols_input])

# Variables numéricas de entrada
numeric_cols_input = ['RoundNumber', 'eventYear', 'meanAirTemp', 'meanTrackTemp',
                      'meanHumid', 'GridPosition', 'CircuitLength', 'designedLaps']
numeric_input = races_info[numeric_cols_input].values
print('Forma de numeric_input:', numeric_input.shape)
print('Forma de encoded_input:', encoded_input.shape)
# Combinar y escalar
X = np.hstack((numeric_input, encoded_input))
scaler_input = MinMaxScaler()
X = scaler_input.fit_transform(X)

Forma de numeric_input: (178, 8)
Forma de encoded_input: (178, 62)


In [283]:
# Codificar variables categóricas de salida

encoder_output = OneHotEncoder(sparse_output=False)
compound_encoded = encoder_output.fit_transform(races_info[['Compound']])

numeric_cols_output = ['lapNumberAtBeginingOfStint', 'StintLen']
numeric_output = races_info[numeric_cols_output].values

scaler_output = MinMaxScaler(feature_range=(0, 1))
numeric_output_scaled = scaler_output.fit_transform(numeric_output)
compound_encoded = encoder_output.fit_transform(races_info[['Compound']])
y = np.hstack((numeric_output_scaled, compound_encoded))

# Arquitectura CGAN

In [284]:
condition_dim = X.shape[1]
num_output_dim = len(numeric_cols_output)  # Número de variables numéricas de salida
cat_output_dim = compound_encoded.shape[1]  # Número de categorías de 'Compound'
strategy_dim = num_output_dim + cat_output_dim
noise_dim = 100

In [285]:
# Generador
def build_generator(noise_dim, condition_dim, num_output_dim, cat_output_dim):
    noise_input = Input(shape=(noise_dim,))
    condition_input = Input(shape=(condition_dim,))
    merged = Concatenate()([noise_input, condition_input])

    x = Dense(128)(merged)
    x = LeakyReLU(0.2)(x)
    x = Dense(256)(x)
    x = LeakyReLU(0.2)(x)

    # Salida numérica
    num_output = Dense(num_output_dim, activation='sigmoid')(x)
    # Salida categórica
    cat_output = Dense(cat_output_dim, activation='softmax')(x)

    # Combinar salidas
    output = Concatenate()([num_output, cat_output])

    model = Model([noise_input, condition_input], output)
    return model

In [286]:
# Discriminador
def build_discriminator(strategy_dim, condition_dim):
    strategy_input = Input(shape=(strategy_dim,))
    condition_input = Input(shape=(condition_dim,))
    merged = Concatenate()([strategy_input, condition_input])

    x = Dense(256)(merged)
    x = LeakyReLU(0.2)(x)
    x = Dense(128)(x)
    x = LeakyReLU(0.2)(x)
    output = Dense(1, activation='sigmoid')(x)

    model = Model([strategy_input, condition_input], output)
    return model

In [287]:
# Compilar modelos
discriminator = build_discriminator(strategy_dim, condition_dim)
discriminator.trainable = True  # Asegúrate de que es entrenable
optimizer_d = Adam(0.0002, 0.5)
discriminator.compile(loss='binary_crossentropy', optimizer=optimizer_d, metrics=['accuracy'])

generator = build_generator(noise_dim, condition_dim, num_output_dim, cat_output_dim)

discriminator.trainable = False
noise_input = Input(shape=(noise_dim,))
condition_input = Input(shape=(condition_dim,))
generated_strategy = generator([noise_input, condition_input])
validity = discriminator([generated_strategy, condition_input])

combined_model = Model([noise_input, condition_input], validity)
optimizer_g = Adam(0.0002, 0.5)
combined_model.compile(loss='binary_crossentropy', optimizer=optimizer_g)

# Entrenamiento del modelo

In [288]:
epochs = 5
batch_size = 32
valid = np.ones((batch_size, 1))
fake = np.zeros((batch_size, 1))

print("Pesos entrenables del discriminador:", len(discriminator.trainable_weights))
print("Pesos entrenables del generador:", len(generator.trainable_weights))
print("Pesos entrenables del modelo combinado:", len(combined_model.trainable_weights))

for epoch in range(epochs):
    # Entrenar el discriminador
    idx = np.random.randint(0, X.shape[0], batch_size)
    real_strategies = y[idx]
    conditions = X[idx]

    noise = np.random.normal(0, 1, (batch_size, noise_dim))
    generated_strategies = generator.predict([noise, conditions])

    d_loss_real = discriminator.train_on_batch([real_strategies, conditions], valid)
    d_loss_fake = discriminator.train_on_batch([generated_strategies, conditions], fake)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # Entrenar el generador
    noise = np.random.normal(0, 1, (batch_size, noise_dim))
    g_loss = combined_model.train_on_batch([noise, conditions], valid)

    # Imprimir progreso
    if epoch % 100 == 0:
        print(f"{epoch} [Pérdida D: {d_loss[0]:.4f}, Precisión: {d_loss[1]*100:.2f}%] [Pérdida G: {g_loss[0]:.4f}]")


Pesos entrenables del discriminador: 0
Pesos entrenables del generador: 8
Pesos entrenables del modelo combinado: 8
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step


C:\Users\naxo_\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\backend\tensorflow\trainer.py:75: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


0 [Pérdida D: 0.7496, Precisión: 31.25%] [Pérdida G: 0.7015]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step


In [289]:
# Generar una nueva estrategia
noise = np.random.normal(0, 1, (1, noise_dim))
sample_condition = X[0:1]  # Puedes elegir cualquier condición existente
generated_strategy = generator.predict([noise, sample_condition])

# Desescalar las variables numéricas
generated_num = generated_strategy[:, :num_output_dim]

generated_num_descaled = scaler_output.inverse_transform(generated_num)

# Decodificar el 'Compound' generado
generated_cat = generated_strategy[:, num_output_dim:]
compound_index = np.argmax(generated_cat, axis=1)
compound_label = encoder_output.categories_[0][compound_index]

print("Estrategia generada:")
print(f"Vuelta de parada: {generated_num_descaled[0][0]:.2f}")
print(f"Duración del stint: {generated_num_descaled[0][1]:.2f}")
print(f"Compuesto de neumático: {compound_label[0]}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
Estrategia generada:
Vuelta de parada: -5.95
Duración del stint: -14.03
Compuesto de neumático: SOFT
